In [1]:
import pandas as pd
import numpy as np

from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()

from string import punctuation
punctuation = punctuation[0:5]+punctuation[7:]
import math
import random

In [3]:
pd.options.display.max_colwidth = 100
discours = pd.read_csv('discours_macron.csv')
# discours[discours['monologue'] == True]

In [6]:
def clean_text(texts):
    texts = texts.apply(lambda x: x.translate(str.maketrans(punctuation, ' '*len(punctuation))))
    texts = texts.apply(lambda x: tokenizer.tokenize(x))
    return texts

def split_sentence(sentence):
    array_sentence = sentence.split(" ")
    return array_sentence

def create_word_index_unigram(texts):
    word_to_index = {
    "START":0,
    "END":1
    }
    index_to_word = ['START', 'END']

    for speech in texts:
        for word in speech:
            if word not in index_to_word:
                word_to_index[word] = len(index_to_word)
                index_to_word.append(word)
                
    return word_to_index, index_to_word

def create_unigram_matrix(word_to_index, index_to_word, texts):
    V = len(index_to_word)
    matrix = np.zeros((V, V))

    for sentence in texts:
        for i in range(len(sentence)):
            if i == 0:
                matrix[0, word_to_index[sentence[i]]] += 1

            else: 
                matrix[word_to_index[sentence[i-1]], word_to_index[sentence[i]]] += 1

        if i == (len(sentence)-1):
            matrix[word_to_index[sentence[i]], 1] += 1

    return matrix


In [81]:

#Process texts
texts = clean_text(discours[discours['monologue'] == True].reset_index().text)

nb_processed_texts = 100

#Get word to index and index to word for unigrams
word_to_index_unigram, index_to_word_unigram = create_word_index_unigram(texts[:nb_processed_texts])

#Get unigram matrix
unigram_matrix = create_unigram_matrix(word_to_index_unigram,  index_to_word_unigram, texts[:nb_processed_texts])

In [82]:
texts

0      [Comme, chaque, 14, Juillet, nous, nous, rassemblons, aujourd'hui, autour, de, nos, armées, et, ...
1      [Françaises, Français, Mes, chers, compatriotes, Je, veux, ce, soir, vous, parler, des, jours, q...
2      [Merci, beaucoup, Madame, la, Chancelière, chère, Angela, MERKEL, En, effet, nous, avons, eu, l'...
3      [Merci, beaucoup, Madame, la, Chancelière, chère, Angela, MERKEL, En, effet, nous, avons, eu, l'...
4      [Un, an, après, le, lancement, à, Paris, de, l'Appel, de, Christchurch, la, France, et, la, Nouv...
                                                      ...                                                 
610    [Mesdames, Messieurs, les, Parlementaires, Mesdames, Messieurs, les, Présidents, les, Présidente...
611    [Bonjour, mes, chers, compatriotes, Messieurs, les, Ministres, Monsieur, l'Ambassadeur, Madame, ...
612    [Ce, qu'il, s'est, passé, aujourd'hui, à, Paris, n'a, rien, à, voir, avec, l'expression, pacifiq...
613    [Mesdames, Messieurs, J'ai, so

In [83]:

def get_next_word(index_previous_ngram, matrix, show_nb_possibilities=False):
    '''
    index_previous_ngram : int, represent the word index in the word_to_index matrix
    matrix : successor matrix, lines represent the n-gram, columns represent the successor

    1. Check how many successors are possible
    2. Select a random word from the possible words, to avoid repetition
    '''
    
    if (matrix[index_previous_ngram]>0).sum() < 2:
        nb_possible = 1
    elif (matrix[index_previous_ngram]>0).sum() > 9:
        nb_possible = 10
    else:
        nb_possible = (matrix[index_previous_ngram]>0).sum()

    if show_nb_possibilities==True:
        print('NUMBER POSSIBLE ', nb_possible)

    top_indexes = matrix[index_previous_ngram].argsort()[-nb_possible:][::-1]
    random_index = math.floor(random.random()*nb_possible)

    index_next_word = top_indexes[random_index]
        
    return index_next_word, matrix[index_previous_ngram][index_next_word]


In [84]:
def get_sentence_unigram(previous_text, word_to_index, index_to_word,  matrix, nb_words_after=100, no_repetition=True):

    used_words = split_sentence(previous_text)
    last_word = used_words[len(used_words)-1]
    print(previous_text, end=" ")
    index_last_word = word_to_index[last_word]
    
    for i in range(nb_words_after):        
        index_last_word, occurence = get_next_word(index_last_word, matrix)
        
        while index_to_word[index_last_word] in used_words:
            index_last_word, occurence = get_next_word(index_last_word, matrix)
                
        used_words.append(index_to_word[index_last_word])
        print(index_to_word[index_last_word], end = ' ')
        if(index_last_word==1):
            break;

get_sentence_unigram("madame", word_to_index_unigram, index_to_word_unigram,unigram_matrix,nb_words_after=100)



madame la fois encore des femmes dans cette salle provisoire de ce qu'on puisse aller plus fort chez vous et décisions le plan économique faire face ces valeurs partagées une réponse très heureux pour nos deux pays République a quelques unes qui dire point région est un peu à projets concrets territoires sont droits Et puis on doit tous les autres se trouve d'abord en France c'est ne fera monde du Fonds mondial forte retraite ministres Mesdames que sécurité commune cela ici prochaines semaines Je met retraites mais il eu C'est aussi nouvelle Président cher OUATTARA tard allaient devenir celle d'un 

In [85]:
index_to_word_unigram[get_next_word(word_to_index_unigram['START'], unigram_matrix)[0]]

'Le'

## BIGRAMS


In [86]:

def create_word_index_bigram(texts):
    word_to_index = {
    "START START":0,
    "END":1
    }
    index_to_word = ['START START', 'END']

    for speech in texts:
        for i in range(len(speech)):
            if i == 0:
                word = 'START ' + speech[0]
            else:
                word = speech[i-1] + ' ' + speech[i]

            if word not in index_to_word:
                word_to_index[word] = len(index_to_word)
                index_to_word.append(word)
            
    return word_to_index, index_to_word

def create_bigram_matrix(word_to_index, index_to_word, word_to_index_unigram, texts):
    V = len(index_to_word)
    V_unigram = len(word_to_index_unigram)
    matrix = np.zeros((V, V_unigram))

    for sentence in texts:
        for i in range(len(sentence)):
            if i == 0:
                matrix[word_to_index['START START'], word_to_index_unigram[sentence[0]]] += 1            
                matrix[word_to_index['START '+sentence[0]], word_to_index_unigram[sentence[1]]] += 1

            elif i >= (len(sentence)-1):
                matrix[word_to_index[sentence[i-1]+' '+sentence[i]], 1] += 1
       
            else: 
                matrix[word_to_index[sentence[i-1]+' '+sentence[i]], word_to_index_unigram[sentence[i+1]]] += 1

    #matrix /= matrix.sum(axis=1, keepdims=False)
    
    return matrix


In [87]:
#Get word to index and index to word for bigrams
word_to_index_bigram, index_to_word_bigram = create_word_index_bigram(texts[:nb_processed_texts])

#Get bigram matrix
bigram_matrix = create_bigram_matrix(word_to_index_bigram, index_to_word_bigram, word_to_index_unigram, texts[:nb_processed_texts])


In [120]:
index_to_word_bigram[:20]

['START START',
 'END',
 'START Comme',
 'Comme chaque',
 'chaque 14',
 '14 Juillet',
 'Juillet nous',
 'nous nous',
 'nous rassemblons',
 "rassemblons aujourd'hui",
 "aujourd'hui autour",
 'autour de',
 'de nos',
 'nos armées',
 'armées et',
 'et nous',
 'nous faisons',
 'faisons corps',
 'corps avec',
 'avec les']

In [89]:
def get_sentence_bigram(previous_text, word_to_index, index_to_word, index_to_word_unigram, matrix, nb_words_after=20, no_repetition=True):

    used_words = split_sentence(previous_text)
    last_bigram = used_words[len(used_words)-2] + ' ' + used_words[len(used_words)-1]
    
    print(previous_text, end=" ")

    index_last_bigram = word_to_index[last_bigram]

    for i in range(nb_words_after):  
        
        index_next_word, occurence = get_next_word(index_last_bigram, matrix)
        
       # while index_to_word[index_last_bigram] in used_words:
       #     index_last_bigram, occurence = get_next_word_bigram(index_last_bigram, matrix)
                
        used_words.append(index_to_word_unigram[index_next_word])
        print(index_to_word_unigram[index_next_word], end = ' ')
        if(index_next_word==1):
            break;
        
        index_last_bigram = word_to_index[used_words[len(used_words)-2] + ' ' + used_words[len(used_words)-1]]
            



In [90]:
idx, occurence = get_next_word(word_to_index_bigram['Mes chers'], bigram_matrix)
print(index_to_word_unigram[idx])
print(occurence)

compatriotes
24.0


In [91]:
get_sentence_bigram("START START", 
                    word_to_index_bigram, 
                    index_to_word_bigram, 
                    index_to_word_unigram, 
                    bigram_matrix,
                   nb_words_after=100
                   )

START START Mes chers compatriotes Depuis quelques semaines Nous tirerons les conséquences concrètes pour vous c'est fait pour le pays Nos associations les travailleurs jusqu'en 2025 ils sont une famille y monter un garage tombé devant sa femme qui aura su bâtir un nouveau partenariat Je pense aux médecins aux infirmiers aux infirmières aux aides soignants aides soignantes qui ont donné leur vie particulière pour cette raison que c'est très cher et d'ailleurs beaucoup mieux dans le nouveau parlement européen qui doit tant à l'engagement pris par le ministre le Ministre ici présent aussi allant des questions à vous de la biodiversité 

## Trigrams

In [92]:

def create_word_index_trigram(texts):
    word_to_index = {
    "START START START":0,
    "END":1
    }
    index_to_word = ['START START START', 'END']

    for speech in texts:
        for i in range(len(speech)):
            if i == 0:
                word = 'START START ' + speech[i]
            elif i==1:
                word = 'START ' + speech[i-1] + ' ' + speech[i]
            else:
                word = speech[i-2] + ' ' + speech[i-1] + ' ' + speech[i]

            #print(word)
            if word not in index_to_word:
                word_to_index[word] = len(index_to_word)
                index_to_word.append(word)
            
    return word_to_index, index_to_word

def create_trigram_matrix(word_to_index, index_to_word, word_to_index_unigram, texts):
    V = len(index_to_word)
    V_unigram = len(word_to_index_unigram)
    matrix = np.zeros((V, V_unigram))

    for sentence in texts:
        for i in range(len(sentence)):
            if i == 0:
                matrix[word_to_index['START START START'], word_to_index_unigram[sentence[0]]] += 1 
                matrix[word_to_index['START START '+sentence[0]], word_to_index_unigram[sentence[i+1]]] += 1 

            elif i ==1:
                matrix[word_to_index['START '+sentence[i-1] + ' ' +sentence[i]], word_to_index_unigram[sentence[i+1]]] += 1

            elif i >= (len(sentence)-1):
                matrix[word_to_index[sentence[i-2] +' '+ sentence[i-1]+' '+sentence[i]], 1] += 1
       
            else: 
                matrix[word_to_index[sentence[i-2]+' '+sentence[i-1]+' '+sentence[i]], word_to_index_unigram[sentence[i+1]]] += 1

    #matrix /= matrix.sum(axis=1, keepdims=False)
    
    return matrix


In [93]:
word_to_index_trigram, index_to_word_trigram = create_word_index_trigram(texts[:nb_processed_texts])

trigram_matrix = create_trigram_matrix(word_to_index_trigram, index_to_word_trigram, word_to_index_unigram, texts[:nb_processed_texts])


In [94]:
unique, counts = np.unique((trigram_matrix!=0).sum(1), return_counts=True)

np.asarray((unique, counts)).T


array([[     0,      1],
       [     1, 243787],
       [     2,  15768],
       [     3,   4377],
       [     4,   1981],
       [     5,    991],
       [     6,    616],
       [     7,    385],
       [     8,    273],
       [     9,    212],
       [    10,    151],
       [    11,    122],
       [    12,     85],
       [    13,     62],
       [    14,     61],
       [    15,     57],
       [    16,     43],
       [    17,     35],
       [    18,     29],
       [    19,     26],
       [    20,      8],
       [    21,     15],
       [    22,     19],
       [    23,     17],
       [    24,      7],
       [    25,     13],
       [    26,     12],
       [    27,     10],
       [    28,     10],
       [    29,      1],
       [    30,      4],
       [    31,      6],
       [    32,      2],
       [    33,      4],
       [    34,      4],
       [    35,      1],
       [    36,      1],
       [    37,      2],
       [    38,      5],
       [    39,      3],


In [96]:
#Proportion of non random selection
(((trigram_matrix!=0).sum(1))==1).sum()*100/trigram_matrix.shape[0]

90.54399863322526

In [97]:
idx, occ = get_next_word(word_to_index_trigram['on va devoir'], trigram_matrix,show_nb_possibilities=True)
print(index_to_word_unigram[idx], occ)

NUMBER POSSIBLE  6
nous 1.0


In [98]:

def get_sentence_trigram(previous_text, word_to_index, index_to_word, index_to_word_unigram, matrix, nb_words_after=20, no_repetition=True):

    used_words = split_sentence(previous_text)
    last_trigram = used_words[len(used_words)-3] + ' ' + used_words[len(used_words)-2] + ' ' + used_words[len(used_words)-1]
    
    print(previous_text, end=" ")

    index_last_trigram = word_to_index[last_trigram]

    for i in range(nb_words_after):  
        
        index_next_word, occurence = get_next_word(index_last_trigram, matrix)

       # while index_to_word[index_last_trigram] in used_words:
       #     index_last_trigram, occurence = get_next_word_bigram(index_last_trigram, matrix)

        used_words.append(index_to_word_unigram[index_next_word])
        print(index_to_word_unigram[index_next_word], end = ' ')
        if(index_next_word==1):
            break;
        
        index_last_trigram = word_to_index[used_words[len(used_words)-3] + ' ' + used_words[len(used_words)-2] + ' ' + used_words[len(used_words)-1]]
            


In [121]:
get_sentence_trigram("START START START", 
                    word_to_index_trigram, 
                    index_to_word_trigram, 
                    index_to_word_unigram, 
                    trigram_matrix,
                   nb_words_after=400)

START START START Françaises Français mes chers compatriotes La réserve sanitaire d'abord avec plus de 40 000 litres jour à 500 000 litres jour à 500 000 litres jour Je veux remercier le ministre de la Culture s'est fortement exprimé sur ce sujet de manière tout à fait le droit d'être moi même le lanceur d'alerte sur ce sujet pour ce qui me concerne je tâcherai de porter en Europe notre voix afin d'avoir plus d'unité et de diversité Eco avait cette formule magnifique en disant la langue de la République il déborde Valeurs actuelles par leur droite ou par leur gauche ce que … GUILLAUME ERNER Ce n'est pas simplement un décret c'est une mobilisation nationale Pour ça j'ai besoin des entreprises pour là aussi faire confiance à ces publics Il y a ensuite l'hospitalité qui fait que vous voir déambuler dans votre ville c'est avoir la conviction que nous poursuivrons notre travail longuement discuté ces dernières semaines avec le débat sur l'immigration qu'il avait souhaité au sortir du grand 